In [136]:
import os
import subprocess
from datetime import datetime


Repositories = {
    'mall-root': 'C:\\Users\\geesi\\Desktop\\BQ\\mall-root',
    'bnq_root': 'C:\\Users\\geesi\\Desktop\\BQ\\bnq_root',
    'bnq_owner_ios': 'C:\\Users\\geesi\\Desktop\\BQ\\bnq_owner_ios',
}

In [139]:
for project_name,git_path in Repositories.items():
    print(project_name, git_path)

mall-root C:\Users\geesi\Desktop\BQ\mall-root
bnq_root C:\Users\geesi\Desktop\BQ\bnq_root
bnq_owner_ios C:\Users\geesi\Desktop\BQ\bnq_owner_ios


In [14]:
os.chdir(Repositories['mall-root'])
result = subprocess.run(['git','log', 'master', 
                            '--pretty=format:"%H,%cn,%ce,%ct"', 
                            '--numstat', '--no-merges'], stdout=subprocess.PIPE)

In [89]:
raw_output = result.stdout
raw_output = raw_output.decode('utf-8')
commit_lines = raw_output.split('\n')

In [134]:
def insert_commit(dbcon, commit, changes):
    with dbcon.cursor() as cursor:
        commit_sql = "INSERT INTO commit (`hash`, `user`, `email`, `commit_date`, `insert`, `delete`, `files`)\
        VALUES (%s, %s, %s, %s, %s, %s, %s)"
        changes_sql = "INSERT INTO file_changes (`commit_id`, `insert`, `delete`, `file_name`)\
        VALUES (%s, %s, %s, %s)"
        
        cursor.execute(commit_sql, commit)
        
        change_values = [[cursor.lastrowid] + c for c in changes]
        cursor.executemany(changes_sql, change_values)
        
        dbcon.commit()

In [135]:
import pymysql
from tqdm import tqdm

def parse_commit_line(commit_line, summary):
    summary[0:4] = commit_line.strip('"').split(',')
    summary[3] = datetime.fromtimestamp(int(summary[3]))
    summary[4:7] = [0, 0, 0]

con = pymysql.connect(host="localhost", user="giter", passwd="giter=01", db="gitwork", charset="utf8")

# {'hash':'', 'user':'', 'email':'', 'commit_date':0, 'insert':0, 'delete':0, 'files':0}
commit_summary = [i for i in range(7)]
changes = []
commit_num = 0
in_analyzing_changes = False

for line in tqdm(commit_lines[0:100]):
    if len(line) == 0:
        if in_analyzing_changes:
            insert_commit(con, commit_summary, changes)
            commit_num += 1
            in_analyzing_changes = False
    else:
        if not in_analyzing_changes:
            parse_commit_line(line, commit_summary)
            in_analyzing_changes = True
            changes = []
        else:
            c = line.split('\t')
            if len(c) == 3:
                c[0] = int(c[0]) if c[0] != '-' else 0
                c[1] = int(c[1]) if c[1] != '-' else 0
                commit_summary[4] += c[0]
                commit_summary[5] += c[0]
                commit_summary[6] += 1
                changes.append(c)
            else:  #In case the previous commit log has no change
                insert_commit(con, commit_summary, changes)
                commit_num += 1
                parse_commit_line(line, commit_summary)
            
if in_analyzing_changes:
    insert_commit(con, commit_summary, changes)
    commit_num += 1
    
print('Total %d commits processed'%commit_num)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 97.33it/s]

Total 26 commits processed


import pymysql
from tqdm import tqdm

con = pymysql.connect(host="localhost", user="giter", passwd="giter=01", db="gitwork", charset="utf8")

i = 0
pbar = tqdm(total=len(all_lines))

while i < len(all_lines):
    old_i = i
    if len(all_lines[i]) != 0:
        hash_, name_, email_, date_ = all_lines[i].strip('"').split(',')
        date_ = datetime.fromtimestamp(int(date_))
        i += 1
        
        all_insert = 0
        all_delete = 0
        all_file = 0
        while len(all_lines[i]) > 0:
            changes = all_lines[i].split('\t')
            if len(changes) == 3:
                insert_, delete_, filename_ = changes
                #print(insert_)
                all_insert += int(insert_) if insert_ != '-' else 0
                all_delete += int(delete_) if delete_ != '-' else 0
                all_file += 1
                
                i += 1
            else:
                break
        insert_commit(con, hash_, name_, email_, date_, all_insert, all_delete, all_file)
        
        pbar.update(i - old_i)
        old_i = i
        
    if len(all_lines[i]) == 0:
        i += 1

con.close()

import pymysql

hash_ = '2349284732987432974'
name_ = u'纪晓茗'
email_ = 'simon@simon.com'
all_insert = 0
all_delete = 0
all_file = 0
date_ = '2018-1-1'

con = pymysql.connect(host="localhost", user="giter", passwd="giter=01", db="gitwork", charset="utf8")

insert_commit(con, hash_, name_, email_, date_, all_insert, all_delete, all_file)